In [1]:
import re
import numpy as np

In [2]:
sentences = []
with open('rytmiczne_zdania_z_korpusu.txt', 'r', encoding='utf8') as f:
    for line in f:
        l = line.rstrip().split(' ', 1)
        sentences.append(l[1])

In [3]:
word_vec_dict = {}
with open('poleval_base_vectors.txt', 'r', encoding='utf8') as f:
    for line in f:
        l = line.split(' ', 1)
        word = l[0]
        vec = [float(x) for x in l[1].split()]
        word_vec_dict[word] = vec

In [4]:
tag_word_dict = {}

with open('supertags.txt', 'r', encoding='utf8') as f:
    for line in f:
        l = line.split()
        if l[1] not in tag_word_dict:
            tag_word_dict[l[1]] = [l[0]]
        else:
            tag_word_dict[l[1]].append(l[0])

In [5]:
word_tag_dict = {}

with open('supertags.txt', 'r', encoding='utf8') as f:
    for line in f:
        l = line.split()
        word_tag_dict[l[0]] = [l[1]]

In [6]:
def regex(text):
    return re.sub(r'[^\w\s]', '', text).rstrip().lower().split()

In [7]:
def find_tags(text):
    tags = []
    for x in regex(text):
        if x in word_tag_dict:
            tags.append(word_tag_dict[x])
        else:
            tags.append(('^' + x)[-3:])
    return tags

In [8]:
def last_syllable(word):
    word = word[::-1]
    vowels = 'aąeęioóuy'
    cnt = 0
    lsyl = ''
    flag = False
    for c in word:
        if c not in vowels:
            lsyl += c
        elif c in vowels and c != 'i':
            flag = True
            cnt += 1
            if cnt < 2:
                lsyl += c
            else:
                lsyl += c
                break
        elif c == 'i' and flag:
            lsyl += c
            flag = False
            continue
        else:
            cnt += 1
            if cnt < 2:
                lsyl += c
            else:
                lsyl += c
                break
    lsyl = lsyl[::-1]
    return lsyl

In [9]:
def count_syllables(word):
    vowels = 'aąeęioóuy'
    cnt = 0
    flag = False
    for c in word:
        if c in vowels and c == 'i':
            flag = True
            cnt += 1
        elif c in vowels and not flag:
            cnt += 1
        else:
            flag = False
            continue
    return cnt      

In [78]:
def rhyme_word(word):
    if word == 'w' or word == 'do' or word == 'z' or word == 'ze' or word == 'na':
        return word
    tag = find_tags(word)[0][0]
    syl_cnt = count_syllables(word)
    lsyl = last_syllable(word)
    candidates = [x for x in tag_word_dict[tag] if x != word and count_syllables(x) == syl_cnt and last_syllable(x) == lsyl]
    if not candidates:
        candidates = [x for x in tag_word_dict[tag] if x != word and count_syllables(x) == syl_cnt]
        if not candidates:
            candidates = [x for x in tag_word_dict[tag] if x != word]
            if not candidates:
                return word
    if word in word_vec_dict:
        vec = word_vec_dict[word]
        vectors = []
        for i in range(len(candidates)):
            if candidates[i] in word_vec_dict and candidates[i] != word:
                vectors.append((i, word_vec_dict[candidates[i]]))
        if not vectors:
            return np.random.choice(candidates, 1)[0]
        products = [(x[0], np.dot(x[1], vec)) for x in vectors]
        max_idx = max(products, key=lambda x: x[1])[0]
        return candidates[max_idx]
    else:
        return np.random.choice(candidates, 1)[0]

In [73]:
def poetize_sentence(sentence):
#     print(f'ORYGINAŁ: {sentence}')
    if np.random.rand() < 0.5:
        first_verse = sentence.split(' [', 1)[0]
        last_word = first_verse.split()[-1]
        modified = rhyme_word(last_word)
        s = sentence.split()
        s[len(s)-2] = modified
    else:
        second_verse = sentence.split('] ', 1)[1]
        last_word = second_verse.split()[-2]
        modified = rhyme_word(last_word)
        s = sentence.split()
        idx = s.index('[*]')
        s[idx-1] = modified
    print('ZMODYFIKOWANA: ' + ' '.join(s))
    print()

In [72]:
for i in range(10):
    sentence = np.random.choice(sentences, 1)[0]
    poetize_sentence(sentence)

ORYGINAŁ: w europie środkowej niektóre gatunki [*] są uprawiane jako rośliny ozdobne .
POEZJA: w europie środkowej niektóre nadobne [*] są uprawiane jako rośliny ozdobne .

ORYGINAŁ: w debacie nad przyszłością polskich stoczni trzeba [*] jednak widzieć nie tylko stoczniowców wybrzeża .
POEZJA: w debacie nad przyszłością polskich stoczni pobrzeża [*] jednak widzieć nie tylko stoczniowców wybrzeża .

ORYGINAŁ: minister spraw wewnętrznych natomiast przestaje [*] być administratorem centralnych rejestrów .
POEZJA: minister spraw wewnętrznych natomiast poliestrów [*] być administratorem centralnych rejestrów .

ORYGINAŁ: na terenie lodowca znajduje się szereg [*] atrakcji dla narciarzy i snowboardzistów .
POEZJA: na terenie lodowca znajduje się szereg [*] atrakcji dla narciarzy i wachlarz .

ORYGINAŁ: decyzja o budowie tarczy nie poprawi [*] wizerunku waszego kraju w europie .
POEZJA: decyzja o budowie tarczy nie poprawi [*] wizerunku waszego kraju w pozbawi .

ORYGINAŁ: po spełnieniu toast

In [74]:
def modify_sentence(sentence):
    print(f'ORYGINAŁ: {sentence}')
    if np.random.rand() < 0.5:
        first_verse = sentence.split(' [', 1)[0]
        words = regex(first_verse)
        res = ''
        for word in words:
            res += rhyme_word(word) + ' '
        r = res.split()
        s = sentence.split()
        idx = s.index('[*]')
        j = 0
        for i in range(0, idx):
            if s[i] == ',' or s[i] == '.' or s[i] == '?' or s[i] == '!' or s[i] == ':':
                continue
            else:
                s[i] = r[j]
                j += 1
    else:
        second_verse = sentence.split('] ', 1)[1]
        words = regex(second_verse)
        res = ''
        for word in words:
            res += rhyme_word(word) + ' '
        r = res.split()
        s = sentence.split()
        idx = s.index('[*]')
        j = 0
        for i in range(idx+1, len(s)):
            if s[i] == ',' or s[i] == '.' or s[i] == '?' or s[i] == '!' or s[i] == ':':
                continue
            else:
                s[i] = r[j]
                j += 1
    modified = ' '.join(s)
    res = poetize_sentence(modified)

In [80]:
for i in range(10):
    sentence = np.random.choice(sentences, 1)[0]
    modify_sentence(sentence)

ORYGINAŁ: matki z dziećmi struchlałe , zaległe po zbożach [*] , i znowu zbrodnia w sercu , a krew lśni na nożach .
ZMODYFIKOWANA: matki z dziećmi struchlałe , zaległe po knotach [*] , i nagle brodnia w skrzelu , a krew dba na botach .

ORYGINAŁ: wstępne prognozy mówią o dwutygodniowej [*] pauzie zawodnika unii hrubieszów .
ZMODYFIKOWANA: wstępne prognozy mówią o dwutygodniowej [*] tuluzie zagrodnika partii fernambukowej .

ORYGINAŁ: możesz zaprogramować dowolne litery [*] , jak również kombinację liter i symboli .
ZMODYFIKOWANA: śmigasz wyregulować bezsolne bristoli [*] , jak również kombinację liter i symboli .

ORYGINAŁ: pani poseł iwona michałek , klub prawo [*] i sprawiedliwość , zada kolejne pytanie .
ZMODYFIKOWANA: pani poseł iwona michałek , klub zwołanie [*] i niezawisłość , nada olejne zabranie .

ORYGINAŁ: piosenka osiągnęła wysokie pozycje [*] na oficjalnych listach w wielu krajach świata .
ZMODYFIKOWANA: sportsmenka podźwignęła głębokie szpata [*] na oficjalnych listach w w